In [1]:
import os
import numpy as np
import matplotlib
import tensorflow as tf
import cv2

# 1. 解决绘图后端问题 (保留你之前的 TkAgg 修复)
matplotlib.rcParams.update(matplotlib.rcParamsDefault)
matplotlib.use('TkAgg', force=True)
import matplotlib.pyplot as plt

# 2. 导入 Waymo 数据定义
try:
    from waymo_open_dataset import dataset_pb2 as open_dataset
    # 尝试导入 E2E 数据定义
    from waymo_open_dataset.protos import end_to_end_driving_data_pb2 as wod_e2ed_pb2
    # ⚠️ 关键：导入官方 Demo 使用的相机投影算子
    from waymo_open_dataset.wdl_limited.camera.ops import py_camera_model_ops
    print(f"✅ 环境加载成功！")
    print(f"   - TensorFlow: {tf.__version__}")
    print(f"   - Camera Ops: 已加载")
except ImportError as e:
    print(f"❌ 导入失败: {e}")
    print("提示: 请确保安装了 waymo-open-dataset-tf-2-12-0 (或兼容版本)")

np.set_printoptions(precision=3, suppress=True)

2026-01-05 17:24:54.891341: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-05 17:24:54.893600: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2026-01-05 17:24:54.932654: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2026-01-05 17:24:54.934259: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-01-05 17:24:55.469085: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:3

✅ 环境加载成功！
   - TensorFlow: 2.13.0
   - Camera Ops: 已加载


In [2]:
# 修改为你的实际文件路径
FILENAME = r'D:\Datasets\WOD_E2E_Camera_v1\val\val_202504211843.tfrecord-00008-of-00093'

# 路径兼容处理 (WSL vs Windows)
if not os.path.exists(FILENAME) and 'mnt' not in FILENAME:
    # 尝试自动转换为 WSL 路径
    FILENAME = FILENAME.replace('D:', '/mnt/d').replace('\\', '/')

print(f"📂 读取文件: {FILENAME}")

try:
    dataset = tf.data.TFRecordDataset(FILENAME, compression_type='')
    dataset_iter = iter(dataset)
    print("✅ 数据集迭代器已就绪")
except Exception as e:
    print(f"❌ 文件读取错误: {e}")

📂 读取文件: /mnt/d/Datasets/WOD_E2E_Camera_v1/val/val_202504211843.tfrecord-00008-of-00093
✅ 数据集迭代器已就绪


2026-01-05 17:24:56.691323: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2026-01-05 17:24:56.692898: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [3]:
def return_front3_cameras(data: wod_e2ed_pb2.E2EDFrame):
    """
    提取前向三个摄像头的数据 (Image) 和 标定参数 (Calibration)
    """
    image_list = []
    calibration_list = []
    
    # 顺序: 左(2) -> 中(1) -> 右(3)
    order = [2, 1, 3] 
    
    # 建立查找表以加快速度
    img_dict = {img.name: img for img in data.frame.images}
    calib_dict = {cal.name: cal for cal in data.frame.context.camera_calibrations}

    for camera_name in order:
        if camera_name in img_dict:
            # 1. 解码图片
            image_content = img_dict[camera_name]
            image = tf.io.decode_image(image_content.image).numpy()
            image_list.append(image)
            
            # 2. 获取对应的标定参数 (这对投影至关重要)
            if camera_name in calib_dict:
                calibration_list.append(calib_dict[camera_name])
            else:
                print(f"⚠️ 警告: 缺少相机 {camera_name} 的标定数据")
                
    return image_list, calibration_list

def project_vehicle_to_image(vehicle_pose, calibration, points):
    """
    将车辆坐标系下的 3D 点投影到 2D 图像平面
    来源: 官方 tutorial_vision_based_e2e_driving.ipynb
    """
    # 1. 构建变换矩阵 (Vehicle -> World)
    pose_matrix = np.array(vehicle_pose.transform).reshape(4, 4)
    
    # 2. 将点从车辆坐标系转到世界坐标系
    # points shape: [N, 3]
    world_points = np.zeros_like(points)
    for i, point in enumerate(points):
        # 齐次坐标变换
        cx, cy, cz, _ = np.matmul(pose_matrix, [*point, 1])
        world_points[i] = (cx, cy, cz)

    # 3. 准备相机参数 Tensor
    extrinsic = tf.reshape(
        tf.constant(list(calibration.extrinsic.transform), dtype=tf.float32),
        [4, 4])
    intrinsic = tf.constant(list(calibration.intrinsic), dtype=tf.float32)
    
    metadata = tf.constant([
        calibration.width,
        calibration.height,
        open_dataset.CameraCalibration.GLOBAL_SHUTTER,
    ], dtype=tf.int32)
    
    # 相机自身的位姿 + 占位符
    camera_image_metadata = list(vehicle_pose.transform) + [0.0] * 10

    # 4. 调用 Waymo 专用算子进行投影
    # 返回: [N, 3] -> (u, v, ok) 其中 ok=1 表示在图像内
    return py_camera_model_ops.world_to_image(
        extrinsic, intrinsic, metadata,
        camera_image_metadata,
        world_points
    ).numpy()

def draw_points_on_image(image, points, color=(255, 0, 0), size=5):
    """在图像上画点"""
    # 复制一份以防修改原图
    canvas = image.copy()
    for point in points:
        u, v, ok = point
        if ok: # 只画在图像范围内的点
            cv2.circle(canvas, (int(u), int(v)), size, color, -1)
    return canvas

In [4]:
try:
    # 1. 取出一帧并解析
    data_bytes = next(dataset_iter)
    data = wod_e2ed_pb2.E2EDFrame()
    data.ParseFromString(data_bytes.numpy())
    
    print(f"📍 当前帧 ID: {data.frame.context.name}")

    # 2. 提取图片和标定
    images, calibs = return_front3_cameras(data)
    
    if len(images) == 3:
        # 3. 提取未来轨迹 (Ground Truth Trajectory)
        # 形状: (N, 3) -> x, y, z
        future_waypoints = np.stack([
            data.future_states.pos_x, 
            data.future_states.pos_y, 
            data.future_states.pos_z
        ], axis=1)
        
        print(f"🚀 提取到未来轨迹点数: {len(future_waypoints)}")

        # 4. 获取车辆当前位姿 (通常第一张图的位姿近似代表车辆位姿)
        vehicle_pose = data.frame.images[0].pose

        # 5. 投影并绘图
        drawn_images = []
        for i in range(3):
            # A. 3D -> 2D 投影
            projected_points = project_vehicle_to_image(
                vehicle_pose, 
                calibs[i], 
                future_waypoints
            )
            # B. 在图上画红点
            img_with_traj = draw_points_on_image(images[i], projected_points, size=15)
            drawn_images.append(img_with_traj)

        # 6. 拼接全景
        panorama = np.concatenate(drawn_images, axis=1)

        # 7. 弹窗显示
        plt.figure(figsize=(20, 8))
        plt.imshow(panorama)
        plt.title(f"E2E Panorama with Future Trajectory (Red Dots)")
        plt.axis('off')
        plt.show()
        print("🎉 可视化完成！请查看弹出的窗口，红点即为车辆的未来真值轨迹。")
        
    else:
        print("⚠️ 本帧数据不完整，无法拼接。")

except StopIteration:
    print("⚠️ 数据已读完，请重新运行 Cell 2。")
except Exception as e:
    print(f"❌ 运行出错: {e}")

📍 当前帧 ID: 5bef8f2e82697e56d4151d1462088b90-206
🚀 提取到未来轨迹点数: 20
🎉 可视化完成！请查看弹出的窗口，红点即为车辆的未来真值轨迹。


In [9]:
import json
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from collections import Counter

# 1. 路径 (WSL 格式)
FILE_PATH = '/mnt/d/Datasets/WOD_E2E_Camera_v1/val/val_sequence_name_to_scenario_cluster.json'

# 解决绘图
matplotlib.rcParams['font.sans-serif'] = ['SimHei', 'DejaVu Sans', 'Arial']
matplotlib.rcParams['axes.unicode_minus'] = False
matplotlib.use('TkAgg', force=True)

print(f"📂 读取文件: {FILE_PATH}")

with open(FILE_PATH, 'r', encoding='utf-8') as f:
    data_mapping = json.load(f)

print(f"✅ 成功加载 JSON，共 {len(data_mapping)} 条记录。")

# ------------------------------------------------------
# 🔬 显微镜模式：看看里面到底长什么样
# ------------------------------------------------------
# 获取第一个 value
first_value = list(data_mapping.values())[0]
print("\n🔍 === 数据结构侦探 ===")
print("每一条数据的值是一个字典，内容如下：")
print(json.dumps(first_value, indent=4))
print("======================\n")

# ------------------------------------------------------
# 🧠 智能提取与统计
# ------------------------------------------------------
# 我们需要从上面的字典里找到代表“类别”的 Key
# 常见的名字可能是: 'cluster', 'type', 'class', 'category'

# 尝试提取值的列表
extracted_values = []

# 自动猜测 Key (你可以根据上面打印的结果手动修改这里)
target_key = None
if isinstance(first_value, dict):
    # 简单的猜测逻辑
    for candidate in ['cluster', 'scenario_cluster', 'type', 'category', 'class_name']:
        if candidate in first_value:
            target_key = candidate
            print(f"💡 自动锁定目标字段: '{target_key}'")
            break
            
    if target_key:
        # 如果找到了，就提取这个字段
        extracted_values = [v.get(target_key, 'Unknown') for v in data_mapping.values()]
    else:
        print("⚠️ 没猜到字段名，将统计整个字典内容的字符串形式...")
        # 兜底方案：把字典转成字符串统计，防止报错
        extracted_values = [str(v) for v in data_mapping.values()]
else:
    # 如果不是字典（应该不会走到这，因为报错说是dict）
    extracted_values = list(data_mapping.values())

# ------------------------------------------------------
# 📊 统计绘图 (复用之前的逻辑)
# ------------------------------------------------------
cluster_counts = Counter(extracted_values)
df = pd.DataFrame.from_dict(cluster_counts, orient='index', columns=['Count'])
df = df.sort_values(by='Count', ascending=False)
total = sum(cluster_counts.values())
df['Percentage'] = (df['Count'] / total * 100).round(2)

print("\n📊 === 统计结果 (Top 10) ===")
print(df.head(10))

plt.figure(figsize=(12, 6))
plot_data = df.head(15)
plt.bar(plot_data.index.astype(str), plot_data['Count'], color='coral', alpha=0.7)
plt.title(f'Scenario Distribution (Key: {target_key if target_key else "Raw Content"})')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

📂 读取文件: /mnt/d/Datasets/WOD_E2E_Camera_v1/val/val_sequence_name_to_scenario_cluster.json
✅ 成功加载 JSON，共 479 条记录。

🔍 === 数据结构侦探 ===
每一条数据的值是一个字典，内容如下：
{
    "scenario_cluster": "Pedestrian"
}

💡 自动锁定目标字段: 'scenario_cluster'

📊 === 统计结果 (Top 10) ===
                       Count  Percentage
Interections             116       24.22
Foreign Object Debris     78       16.28
Cyclist                   71       14.82
Pedestrian                52       10.86
Multi-Lane Maneuvers      42        8.77
Single-Lane Maneuvers     38        7.93
Special Vehicles          25        5.22
Others                    22        4.59
Cut_ins                   20        4.18
Construction              15        3.13


In [ ]:
# import cv2
# import numpy as np
# import tensorflow as tf

# # 1. 准备工作
# OUTPUT_VIDEO_NAME = "waymo_demo_video.mp4"
# FPS = 10 # Waymo 数据通常是 10Hz
# video_writer = None

# print(f"🎬 开始生成视频，目标文件: {OUTPUT_VIDEO_NAME}")
# print("⏳这可能需要一两分钟，请耐心等待...")

# # 重新加载数据集（确保从头开始）
# try:
#     dataset = tf.data.TFRecordDataset(FILENAME, compression_type='')
    
#     # --- 核心循环开始 ---
#     # 这次我们不用 next()，而是用 for 循环遍历所有数据
#     for i, data_bytes in enumerate(dataset):
        
#         # A. 解析数据
#         frame = wod_e2ed_pb2.E2EDFrame()
#         frame.ParseFromString(data_bytes.numpy())
        
#         # B. 提取图片和标定
#         # (复用之前定义的 return_front3_cameras 函数)
#         images, calibs = return_front3_cameras(frame)
        
#         if len(images) == 3:
#             # C. 提取并投影轨迹 (复用之前逻辑)
#             future_waypoints = np.stack([
#                 frame.future_states.pos_x, 
#                 frame.future_states.pos_y, 
#                 frame.future_states.pos_z
#             ], axis=1)
            
#             vehicle_pose = frame.frame.images[0].pose
            
#             drawn_images = []
#             for cam_idx in range(3):
#                 # 投影
#                 proj_points = project_vehicle_to_image(
#                     vehicle_pose, calibs[cam_idx], future_waypoints
#                 )
#                 # 画点
#                 img_drawn = draw_points_on_image(images[cam_idx], proj_points, size=15)
#                 drawn_images.append(img_drawn)
            
#             # D. 拼接全景
#             panorama = np.concatenate(drawn_images, axis=1)
            
#             # E. 视频写入器的初始化 (只在第一帧做一次)
#             if video_writer is None:
#                 h, w, c = panorama.shape
#                 # OpenCV 视频编码器设置
#                 fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
#                 video_writer = cv2.VideoWriter(OUTPUT_VIDEO_NAME, fourcc, FPS, (w, h))
#                 print(f"   🎥 视频尺寸已确认为: {w}x{h}")

#             # F. 颜色转换并写入
#             # Matplotlib 是 RGB，OpenCV 是 BGR。必须转换，否则视频颜色发蓝。
#             panorama_bgr = cv2.cvtColor(panorama, cv2.COLOR_RGB2BGR)
#             video_writer.write(panorama_bgr)
            
#             # 每处理 20 帧打印一次进度
#             if i % 20 == 0:
#                 print(f"   ...已处理 {i} 帧")

#     # --- 循环结束 ---
    
#     # 释放资源
#     if video_writer:
#         video_writer.release()
#         print(f"\n✅ 视频生成完毕！保存为: {OUTPUT_VIDEO_NAME}")
#         print(f"   总共处理帧数: {i+1}")
#     else:
#         print("❌ 未能生成视频，可能是因为第一帧就没读到数据。")

# except Exception as e:
#     print(f"❌ 处理过程中出错: {e}")
#     if video_writer:
#         video_writer.release()